# Descripción del caso
En este estudio, se recopilaron datos transversales de las Encuestas demográficas y de salud de Nigeria (NDHS) de 2018 para responder a preguntas de investigación sobre el efecto de la edad de las madres y otros factores socioeconómicos en el nivel de anemia de los niños de 0 a 59 meses en Nigeria. Las DHS son encuestas transversales de hogares representativas a nivel nacional que generalmente se realizan cada 5 años.
Los datos de esta encuesta consideraron los 36 estados de Nigeria, así como el Territorio de la Capital Federal (FCT). La población objetivo de este estudio son los niños de 0 a 59 meses y las madres de 15 a 49 años.
En esta encuesta, el ingreso del hogar se midió utilizando el índice de riqueza, la edad actual en grupos de 5 años se produce agrupando la edad actual en años completados, tipo de lugar de residencia donde el encuestado fue entrevistado como urbano o rural, la categorización se creó en función de si el número de punto de muestra o conglomerado se define como urbano o rural, el nivel más alto de educación alcanzado es una variable estandarizada que proporciona el nivel de educación en las siguientes categorías: Sin educación, Educación primaria, secundaria y superior, el número total de nacimientos en los últimos cinco años se define como todos los nacimientos en los meses 0 a 59 anteriores al mes de la entrevista, donde el mes 0 es el mes de la entrevista, la edad del encuestado en el primer nacimiento se calcula utilizando el CMC de la fecha de nacimiento del encuestado.

Después de la depuración de los datos, se utilizó el método Chi cuadrado para probar las hipótesis sobre la posible relación que existe entre ciertos factores socioeconómicos y los niveles de anemia en niños de 0 a 59 meses. El nivel de anemia fue la variable predictora y las variables explicativas son la edad de la madre, el nivel de educación, el índice de riqueza, el nacimiento en los últimos cinco años, el uso de mosquiteros, etc.


# Diccionario de datos
Type of place of residence
```Python
0: Rural
1: Urban


Highest educational level
0: Higher
1: No education
2: Primary
3: Secondary


Wealth index combined
0: Middle
1: Poorer
2: Poorest
3: Richer
4: Richest


Anemia level
0: Mild
1: Moderate
2: Not anemic
3: Severe


Have mosquito bed net for sleeping (from household questionnaire)
0: No
1: Yes


Smokes cigarettes
0: No
1: Yes


Current marital status
0: Divorced
1: Living with partner
2: Married
3: Never in union
4: No longer living together/separated
5: Widowed


Currently residing with husband/partner
0: Living with her
1: Staying elsewhere


When child put to breast
0: 102.0
1: 103.0
2: 104.0
... (continuando con valores similares)
38: Days: 1
39: Hours: 1
40: Immediately


Had fever in last two weeks
0: Don't know
1: No
2: Yes


Taking iron pills, sprinkles or syrup
0: Don't know
1: No
2: Yes
```

## 1. Implementacion de funciones para limpiar, ordenar y transformar los datos.

In [16]:
import pandas as pd

# Cargar el archivo proporcionado por el usuario
file_path = 'children anemia.csv'
data = pd.read_csv(file_path)

# Mostrar una vista previa de los datos para analizar la estructura
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33924 entries, 0 to 33923
Data columns (total 17 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Age in 5-year groups                                                   33924 non-null  object 
 1   Type of place of residence                                             33924 non-null  object 
 2   Highest educational level                                              33924 non-null  object 
 3   Wealth index combined                                                  33924 non-null  object 
 4   Births in last five years                                              33924 non-null  int64  
 5   Age of respondent at 1st birth                                         33924 non-null  int64  
 6   Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)  13136 non-null 

(  Age in 5-year groups Type of place of residence Highest educational level  \
 0                40-44                      Urban                    Higher   
 1                35-39                      Urban                    Higher   
 2                25-29                      Urban                    Higher   
 3                25-29                      Urban                 Secondary   
 4                20-24                      Urban                 Secondary   
 
   Wealth index combined  Births in last five years  \
 0               Richest                          1   
 1               Richest                          1   
 2               Richest                          1   
 3               Richest                          1   
 4               Richest                          1   
 
    Age of respondent at 1st birth  \
 0                              22   
 1                              28   
 2                              26   
 3                              25

In [17]:

# Funciones para limpiar, ordenar y transformar los datos
def limpiar_datos(dataframe):
    """
    Limpia los datos eliminando columnas duplicadas, renombrando columnas y gestionando valores faltantes.
    """
    # Eliminar columnas duplicadas o irrelevantes
    columnas_a_eliminar = ['Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)', 
                           'Anemia level.1']
    dataframe = dataframe.drop(columns=columnas_a_eliminar, errors='ignore')

    # Renombrar columnas para mayor claridad
    dataframe = dataframe.rename(columns={
        'Age in 5-year groups': 'Age_Group',
        'Type of place of residence': 'Residence_Type',
        'Highest educational level': 'Education_Level',
        'Wealth index combined': 'Wealth_Index',
        'Births in last five years': 'Births_Last_5_Years',
        'Age of respondent at 1st birth': 'Age_First_Birth',
        'Anemia level': 'Anemia_Level',
        'Have mosquito bed net for sleeping (from household questionnaire)': 'Mosquito_Net',
        'Smokes cigarettes': 'Smokes',
        'Current marital status': 'Marital_Status',
        'Currently residing with husband/partner': 'Residing_With_Partner',
        'When child put to breast': 'Breastfeeding_Timing',
        'Had fever in last two weeks': 'Fever_Last_2_Weeks',
        'Hemoglobin level adjusted for altitude (g/dl - 1 decimal)': 'Hemoglobin_Level',
        'Taking iron pills, sprinkles or syrup': 'Iron_Supplements'
    })

    # Manejo de valores faltantes
    dataframe['Anemia_Level'] = dataframe['Anemia_Level'].fillna('Unknown')  # Llenar valores faltantes de anemia con "Unknown"
    dataframe = dataframe.dropna(subset=['Hemoglobin_Level', 'Education_Level'])  # Eliminar filas con valores críticos faltantes

    return dataframe


def transformar_datos(dataframe):
    """
    Transforma los datos categóricos a variables numéricas y estandariza las columnas.
    """
    # Convertir categorías a valores numéricos
    categoricas_a_codificar = ['Residence_Type', 'Education_Level', 'Wealth_Index', 'Anemia_Level',
                               'Mosquito_Net', 'Smokes', 'Marital_Status', 'Residing_With_Partner',
                               'Breastfeeding_Timing', 'Fever_Last_2_Weeks', 'Iron_Supplements']

    for columna in categoricas_a_codificar:
        dataframe[columna] = dataframe[columna].astype('category').cat.codes

    return dataframe


# Cargar el archivo CSV
file_path = 'children anemia.csv'  # Reemplaza con la ruta del archivo en tu sistema
data = pd.read_csv(file_path)

# Aplicar funciones al dataset
datos_limpios = limpiar_datos(data)
datos_transformados = transformar_datos(datos_limpios)

# Mostrar las primeras filas del DataFrame transformado
print(datos_transformados.head())

# Guardar los datos transformados en un archivo CSV
datos_transformados.to_csv('datos_limpios_transformados.csv', index=False)
print("Datos guardados como 'datos_limpios_transformados.csv'.")


   Age_Group  Residence_Type  Education_Level  Wealth_Index  \
3      25-29               1                3             4   
5      30-34               1                0             4   
6      35-39               1                3             4   
9      20-24               1                3             4   
12     25-29               1                0             4   

    Births_Last_5_Years  Age_First_Birth  Anemia_Level  Mosquito_Net  Smokes  \
3                     1               25             1             1       0   
5                     1               30             0             1       0   
6                     2               32             2             1       0   
9                     1               19             1             1       0   
12                    1               24             0             1       0   

    Marital_Status  Residing_With_Partner  Breastfeeding_Timing  \
3                2                      0                     3   
5     

## 2 Calcular métricas como media, mediana, moda, y otras estadísticas relevantes

In [18]:
def calcular_metricas_avanzadas(dataframe):
    """
    Calcula métricas descriptivas avanzadas como percentiles, asimetría, curtosis y más.
    """
    # Seleccionar columnas numéricas
    columnas_numericas = dataframe.select_dtypes(include=['int64', 'float64'])

    # Crear un diccionario para almacenar las métricas
    estadisticas = {}

    for columna in columnas_numericas.columns:
        estadisticas[columna] = {
            'Media': columnas_numericas[columna].mean(),
            'Mediana': columnas_numericas[columna].median(),
            'Moda': columnas_numericas[columna].mode().iloc[0] if not columnas_numericas[columna].mode().empty else None,
            'Desviación Estándar': columnas_numericas[columna].std(),
            'Mínimo': columnas_numericas[columna].min(),
            'Máximo': columnas_numericas[columna].max(),
            'Rango': columnas_numericas[columna].max() - columnas_numericas[columna].min(),
            'Percentil 25': columnas_numericas[columna].quantile(0.25),
            'Percentil 75': columnas_numericas[columna].quantile(0.75),
            'Asimetría': columnas_numericas[columna].skew(),
            'Curtosis': columnas_numericas[columna].kurt(),
            'Rango Intercuartílico (IQR)': columnas_numericas[columna].quantile(0.75) - columnas_numericas[columna].quantile(0.25),
            'Coeficiente de Variación (CV)': columnas_numericas[columna].std() / columnas_numericas[columna].mean(),
        }

    # Convertir a un DataFrame para mejor visualización
    estadisticas_df = pd.DataFrame(estadisticas).transpose()
    return estadisticas_df


# Calcular métricas avanzadas
metricas_avanzadas = calcular_metricas_avanzadas(datos_transformados)

# Mostrar las métricas avanzadas
print(metricas_avanzadas)


                          Media  Mediana   Moda  Desviación Estándar  Mínimo  \
Births_Last_5_Years    1.776075      2.0    2.0             0.678971     1.0   
Age_First_Birth       19.958751     19.0   17.0             4.440603    12.0   
Hemoglobin_Level     101.270183    103.0  102.0            15.569583    29.0   

                     Máximo  Rango  Percentil 25  Percentil 75  Asimetría  \
Births_Last_5_Years     6.0    5.0           1.0           2.0   0.520783   
Age_First_Birth        45.0   33.0          17.0          22.0   0.970134   
Hemoglobin_Level      170.0  141.0          92.0         112.0  -0.448152   

                     Curtosis  Rango Intercuartílico (IQR)  \
Births_Last_5_Years  0.160659                          1.0   
Age_First_Birth      1.116339                          5.0   
Hemoglobin_Level     0.579912                         20.0   

                     Coeficiente de Variación (CV)  
Births_Last_5_Years                       0.382287  
Age_First_Birth

## 3. Estructuras de datos (listas, pilas, colas) que faciliten el análisis 

### 3.1 Lista
Las listas son ideales para almacenar datos tabulares o registros específicos que se necesitan procesar en secuencia.


In [10]:
# Extraer datos relevantes y almacenarlos en una lista
lista_anemia = datos_transformados[['Age_Group', 'Anemia_Level', 'Hemoglobin_Level']].values.tolist()

# Ejemplo de acceso a los datos
print("Ejemplo de registros en lista:")
print(lista_anemia[:5])  # Imprime los primeros 5 registros


Ejemplo de registros en lista:
[['25-29', 1, 114.0], ['30-34', 0, 119.0], ['35-39', 2, 102.0], ['20-24', 1, 113.0], ['25-29', 0, 109.0]]


### 3.2 Pila (Stack)
Las pilas siguen el principio LIFO (Last In, First Out) y son útiles si los datos se necesitan procesar en orden inverso.

In [11]:
class Pila:
    def __init__(self):
        self.stack = []

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        return self.stack.pop() if not self.is_empty() else None

    def peek(self):
        return self.stack[-1] if not self.is_empty() else None

    def is_empty(self):
        return len(self.stack) == 0

# Crear una pila con los datos relevantes
pila_anemia = Pila()
for _, row in datos_transformados.iterrows():
    pila_anemia.push({'Age_Group': row['Age_Group'], 'Anemia_Level': row['Anemia_Level']})

# Ejemplo de uso de la pila
print("Dato extraído de la pila:", pila_anemia.pop())


Dato extraído de la pila: {'Age_Group': '20-24', 'Anemia_Level': 2}


### 3.3 Cola (Queue)
Las colas siguen el principio FIFO (First In, First Out) y son útiles si los datos se deben procesar en el mismo orden en que se almacenaron.

In [12]:
from collections import deque

# Crear una cola con los datos relevantes
cola_anemia = deque()
for _, row in datos_transformados.iterrows():
    cola_anemia.append({'Age_Group': row['Age_Group'], 'Anemia_Level': row['Anemia_Level']})

# Ejemplo de uso de la cola
print("Dato extraído de la cola:", cola_anemia.popleft())


Dato extraído de la cola: {'Age_Group': '25-29', 'Anemia_Level': 1}
